# Chemistry example: Cement

This example is about chemical components. The idea is to take input from a natural language text that contains descriptions about chemical components. With Natural Language Processing the relevant component names shall be extracted and the corresponding components shall be looked up in the Wikidata knowledge graph.

See also:
[Entity Linking](https://rq.bitplan.com/index.php/Entity_Linking)

## Example
Input as natural text: "Polylactic acid (PLA) and Acrylonitrile butadiene styrene (ABS) are 3D printing filaments."
Extraction of component names:

* PLA - Polylactic acid
* ABS - Acrylonitrile butadiene styrene

Lookup in Wikidata as:
* [PLA - Q413769](https://www.wikidata.org/wiki/Q413769) (C₃H₆O₃)x
* [ABS - Q143496](https://www.wikidata.org/wiki/Q143496) C₁₅H₁₇N

# Prerequisites
Please run the prerequiste cells before trying the examples.

## Install python module dependencies

In [1]:
import sys
def installModule(projectName:str, moduleName:str=None):
    '''Installs and loads the given module if not already installed'''
    if moduleName is None:
        moduleName=projectName
    !python -m pip install -U --no-input $projectName
    print(f'{projectName} installed')
    %reload_ext $moduleName

installModule('jupyter-xml')
installModule('SPARQLWrapper')
installModule('tabulate')
installModule('spacy==3.1.3', 'spacy')

jupyter-xml installed


SPARQLWrapper installed
tabulate installed
spacy==3.1.3 installed


## Download Models

In [2]:
!python -m spacy download en_core_web_sm    # efficent

     |████████████████████████████████| 13.6 MB 4.9 MB/s eta 0:00:01    |████                            | 1.7 MB 3.2 MB/s eta 0:00:04
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 3.0.0
    Uninstalling en-core-web-sm-3.0.0:
      Successfully uninstalled en-core-web-sm-3.0.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


# Chemistry Example Wikidata Query
see [pyLodStorage Random Substances with CAS number example](http://wiki.bitplan.com/index.php/PyLoDStorage#15_Random_substances_with_CAS_number)

## Extract text from website
We take input from a 
[Penn State University text about the composition of Cement](https://www.engr.psu.edu/ce/courses/ce584/concrete/library/construction/curing/Composition%20of%20cement.htm)
which contains mentions of compounds such as "Silicon dioxide". We'd like to lookup the corresponding Wikidata entry [Silicon dioxide: Q116269](https://www.wikidata.org/wiki/Q116269)

In [3]:
from newspaper import Article
url="https://www.engr.psu.edu/ce/courses/ce584/concrete/library/construction/curing/Composition%20of%20cement.htm"
article = Article(url)
article.download()
article.parse()
text=article.text
print(text[:149],"...")

Composition of cement

Introduction

Portland cement gets its strength from chemical reactions between the cement and water. The process is known as  ...


# NLP with spacy
Try to identify [chemical compounds](https://www.wikidata.org/wiki/Q11173) with the natural language processing library [Spacy](https://spacy.io/)

In [4]:
import spacy
from tabulate import tabulate
# Load English tokenizer, tagger, parser and NER
nlp = spacy.load('en_core_web_sm')
doc = nlp(text)
nouns=[chunk.text for chunk in doc.noun_chunks]

print(f"Found nouns:\n {nouns}")

foundEntities=[{"Text":entity.text, "Entity Tag":entity.label_} for entity in doc.ents]
print(tabulate(foundEntities, headers="keys"))

Found nouns:
 ['Composition', 'cement', 'Portland cement', 'its strength', 'chemical reactions', 'the cement', 'water', 'The process', 'hydration', 'a complex process', 'the chemical composition', 'cement', 'Manufacture', 'cement', 'Portland cement', 'the following materials', 'Lime or calcium oxide', 'CaO', 'limestone', 'chalk', 'shells', 'rock\n\n\n\nSilica', 'SiO', 'sand', 'sand', 'old bottles', 'clay', 'argillaceous rock', 'Alumina', 'Al 2 O', 'bauxite', 'recycled aluminum', 'bauxite', 'recycled aluminum', 'clay', 'clay', 'iron ore', 'scrap iron', 'ash', 'clay', 'iron ore', 'scrap iron', 'ash', ', CaSO', 'limestone', 'Chemical shorthand', 'the complex chemical nature', 'cement', 'a shorthand form', 'the chemical compounds', 'The shorthand', 'the basic compounds', '\n\n\n\nCompound Formula Shorthand form Calcium oxide', '(lime', 'C Silicon dioxide', 'silica', 'SiO 2 S Aluminum oxide', 'alumina', 'Al 2 O', 'A Iron oxide', 'Fe 2 O 3 F Water H', '2 O H Sulfate', '3 S', 'Compound Formul

# Query wikidata for mentioned Chemical Compounds
The NER (Named Entity Recognition) of Spacy seems not to detect the chemical compounds as a known category. 
Thus we need to work around this limitation and try to use the found nouns to query wikidata for the referenced compounds.

## Setting up the [wikidata query endpoint](https://query.wikidata.org/)

In [5]:
from lodstorage.query import QueryManager, Query
from lodstorage.sparql import SPARQL
from IPython.display import display, Markdown
endpoint=SPARQL("https://query.wikidata.org/sparql")

## Get matching entites
The found nouns by spacy can be matched against the labels of chemical compounds.

A wikidata entity such as [Silicon dioxide: Q116269](https://www.wikidata.org/wiki/Q116269) has [rdfs:label](https://www.w3.org/TR/rdf-schema/#ch_label) for different languages specified with the languages tag (e.g. @en). 

To also query the names that are defined in the __Also known as__ column of the entity page the [skos:altLabel](https://www.w3.org/2009/08/skos-reference/skos.html#altLabel) property has to be queried. Searching for the label in both properties can be accomplished by using the [alternative path](https://www.w3.org/TR/sparql11-query/#propertypaths) feature of SPARQL.

Note: The wikidata endpoint has a horizontal line limit and thus the values in the [VALUES clause](https://www.w3.org/TR/sparql11-query/#inline-data) are added over multiple lines.


In [41]:
from SPARQLWrapper import SPARQLWrapper, JSON, CSV
from tabulate import tabulate

nouns=[noun.replace("\n"," ").strip() for noun i]
# building the query string
values=[' '.join([f'"{noun}"@en' for noun in nouns[n*50:n*50+50]]) for n in range((len(nouns)//50)+1)]
print(len(values))
queryValueFormatTags='%s\n'*(len(values))
substanceQueryStr = """
SELECT DISTINCT ?substance
WHERE { 
  VALUES ?substanceLabel {%s}
  ?substance wdt:P31 wd:Q11173;
             rdfs:label|skos:altLabel ?substanceLabel.
}
LIMIT 50
""" 
substanceQueryStr = substanceQueryStr % ("\n".join(values))
print(substanceQueryStr)
# executing the query
substanceQuery=Query(query=substanceQueryStr,
            name="Recognized chemical compounds",
            lang="sparql")
substanceQueryResLoD=endpoint.queryAsListOfDicts(substanceQuery.query)

substances=[record.get('substance') for record in substanceQueryResLoD]

# pretty printout of the result
doc=substanceQuery.documentQueryResult(substanceQueryResLoD, tablefmt="github",floatfmt=".0f",tryItUrl="https://query.wikidata.org")
display(Markdown(str(doc)))

SyntaxError: f-string expression part cannot include a backslash (<ipython-input-41-d5df47650664>, line 5)

## Extract additional information about the entities
The queried substances can now be used to query additional information about them. Since some information might not be avaliable for a substance, the prperties are queried in an [OPTIONAL clause](https://www.w3.org/TR/sparql11-query/#optionals) to include the entity eventhough the triple match is not fulfilled.

In [ ]:
substanceDetailQueryStr = """
SELECT DISTINCT ?substance ?substanceLabel ?formula ?structure ?CAS
WHERE { 
  VALUES ?substance { %s }
  ?substance wdt:P31 wd:Q11173.
  OPTIONAL{
      ?substance wdt:P231 ?CAS.
  }
  OPTIONAL{
      ?substance wdt:P274 ?formula.
  }
  OPTIONAL{
      ?substance wdt:P117  ?structure.
  }
  OPTIONAL{
      ?substance rdfs:label ?substanceLabel.
      FILTER(lang(?substanceLabel)="en")
  }                    
}
LIMIT 50

""" % " ".join([f"<{substance}>" for substance in substances])

substanceDetailQuery=Query(query=substanceDetailQueryStr,
            name="Chemical compound details",
            lang="sparql")
substanceDetailQueryResLoD=endpoint.queryAsListOfDicts(substanceDetailQuery.query)

# pretty printout of the result
doc=substanceDetailQuery.documentQueryResult(substanceDetailQueryResLoD, tablefmt="github",floatfmt=".0f",tryItUrl="https://query.wikidata.org")
display(Markdown(str(doc)))